In [1]:
import tomotopy as tp
import pandas as pd

In [2]:
df = pd.read_json(r'C:\Users\jzmuj\OneDrive\Pulpit\uczelnia\topic modelling\dataset.json').T

In [3]:
df

,text,citations
Machine Learning for Sociology,"SO45CH26_Garip\nARjats.cls\nMay 3, 2019\n12:9\...",169.0
Machine-learning approaches in drug discovery methods and applications,Please cite this article in press as: Lavecchi...,585.0
MLbase A Distributed Machine-learning System,MLbase: A Distributed Machine-learning System\...,444.0
Machine learning and software engineering,"Software Quality Journal, 11, 87–119, 2003\n ...",214.0
A survey of multi-view machine learning,INVITED REVIEW\nA survey of multi-view machine...,829.0
mlr Machine Learning in R,Journal of Machine Learning Research 17 (2016)...,720.0
Machine learning,Annu. Rev. Comput. Sci. 1990. 4:255-306 \nCopy...,211.0
Automatic differentiation in machine learning a survey,Journal of Machine Learning Research 18 (2018)...,1975.0
Argument based machine learning,Artiﬁcial Intelligence 171 (2007) 922–937\nwww...,170.0
A Machine Learning Approach to Workflow Management,A Machine Learning Approach to Workﬂow\nManage...,80.0


In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.porter import PorterStemmer
import re
from nltk.tokenize import RegexpTokenizer
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_text(text: str):
    return ' '.join([stemmer.stem(word) for word in tokenizer.tokenize(' '.join([re.sub(r'\s+', ' ', word) for word in text.split()]))])


In [5]:
df['text_pp'] = df['text'].apply(lambda x: preprocess_text(x))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(ngram_range = (1, 3),
                                max_df = 0.75,
                                min_df = 0.01,
                                tokenizer = tokenizer.tokenize,
                                stop_words='english'
)

In [7]:
df.head()

,text,citations,text_pp
Machine Learning for Sociology,"SO45CH26_Garip\nARjats.cls\nMay 3, 2019\n12:9\...",169.0,so45ch26_garip arjat cl may 3 2019 12 9 annual...
Machine-learning approaches in drug discovery methods and applications,Please cite this article in press as: Lavecchi...,585.0,pleas cite thi articl in press as lavecchia a ...
MLbase A Distributed Machine-learning System,MLbase: A Distributed Machine-learning System\...,444.0,mlbase a distribut machin learn system tim kra...
Machine learning and software engineering,"Software Quality Journal, 11, 87–119, 2003\n ...",214.0,softwar qualiti journal 11 87 119 2003 2003 kl...
A survey of multi-view machine learning,INVITED REVIEW\nA survey of multi-view machine...,829.0,invit review a survey of multi view machin lea...


In [8]:
tf = tf_vectorizer.fit_transform(df['text_pp'])

In [9]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
params = {
    'n_components': list(range(5, 150, 5)),
    'learning_decay': np.arange(0.6, 0.9, 0.1),
    'learning_offset': np.arange(10, 150, 10),
    'perp_tol': [0.1],
    'max_iter': [20],
    'mean_change_tol': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'max_doc_update_iter': [100]
}
lda = LatentDirichletAllocation()
rsearch = RandomizedSearchCV(lda, params, n_iter=30, verbose=3, cv=3, error_score='raise')
rsearch.fit(tf)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV 1/3] END learning_decay=0.7999999999999999, learning_offset=90, max_doc_update_iter=100, max_iter=20, mean_change_tol=1e-06, n_components=65, perp_tol=0.1;, score=-23257697.852 total time= 1.3min
[CV 2/3] END learning_decay=0.7999999999999999, learning_offset=90, max_doc_update_iter=100, max_iter=20, mean_change_tol=1e-06, n_components=65, perp_tol=0.1;, score=-21658335.713 total time= 1.2min
[CV 3/3] END learning_decay=0.7999999999999999, learning_offset=90, max_doc_update_iter=100, max_iter=20, mean_change_tol=1e-06, n_components=65, perp_tol=0.1;, score=-14654811.774 total time= 1.3min
[CV 1/3] END learning_decay=0.8999999999999999, learning_offset=80, max_doc_update_iter=100, max_iter=20, mean_change_tol=0.01, n_components=145, perp_tol=0.1;, score=-46540642.898 total time= 2.9min
[CV 2/3] END learning_decay=0.8999999999999999, learning_offset=80, max_doc_update_iter=100, max_iter=20, mean_change_tol=0.01, n_component

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=LatentDirichletAllocation(), n_iter=30,
                   param_distributions={'learning_decay': array([0.6, 0.7, 0.8, 0.9]),
                                        'learning_offset': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140]),
                                        'max_doc_update_iter': [100],
                                        'max_iter': [20],
                                        'mean_change_tol': [1e-06, 1e-05,
                                                            0.0001, 0.001,
                                                            0.01],
                                        'n_components': [5, 10, 15, 20, 25, 30,
                                                         35, 40, 45, 50, 55, 60,
                                                         65, 70, 75, 80, 85, 90,
                                                         95, 100, 105, 110, 115,
                                                         120, 125, 130, 135,
                                                         140, 145],
                                        'perp_tol': [0.1]},
                   verbose=3)

In [11]:
best = rsearch.best_estimator_

In [13]:
best.perplexity(tf)

344173.8171033977

In [14]:
import pickle
pickle.dump(best, open('model.pkl', 'wb'))